In [171]:
import pandas as pd
import re
import string
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [172]:
file_name = 'data.csv' 
tweet_data = pd.read_csv(file_name, encoding='latin-1')

### EDA

In [173]:
tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [174]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet          13169 non-null  object
 1   HS             13169 non-null  int64 
 2   Abusive        13169 non-null  int64 
 3   HS_Individual  13169 non-null  int64 
 4   HS_Group       13169 non-null  int64 
 5   HS_Religion    13169 non-null  int64 
 6   HS_Race        13169 non-null  int64 
 7   HS_Physical    13169 non-null  int64 
 8   HS_Gender      13169 non-null  int64 
 9   HS_Other       13169 non-null  int64 
 10  HS_Weak        13169 non-null  int64 
 11  HS_Moderate    13169 non-null  int64 
 12  HS_Strong      13169 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.3+ MB


In [175]:
tweet_data.describe()

,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
count,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000
mean,0.422280,0.382945,0.271471,0.150809,0.060217,0.042980,0.024527,0.023236,0.284000,0.256891,0.129471,0.035918
std,0.493941,0.486123,0.444735,0.357876,0.237898,0.202819,0.154685,0.150659,0.450954,0.436935,0.335733,0.186092
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [176]:
tweet_data.dtypes

Tweet            object
HS                int64
Abusive           int64
HS_Individual     int64
HS_Group          int64
HS_Religion       int64
HS_Race           int64
HS_Physical       int64
HS_Gender         int64
HS_Other          int64
HS_Weak           int64
HS_Moderate       int64
HS_Strong         int64
dtype: object

## DATA CLEANING & PREPROCESSING

In [177]:
Duplicated_row = tweet_data.duplicated().sum()
print ("Jumlah duplikat data terdeteksi:", Duplicated_row)

Jumlah duplikat data terdeteksi: 125


In [178]:
tweet_data_cleaned = tweet_data.drop_duplicates()
print("Jumlah duplikat data setelah dihapus:", tweet_data_cleaned.duplicated().sum())

Jumlah duplikat data setelah dihapus: 0


In [179]:
print(tweet_data.isnull().sum())

Tweet            0
HS               0
Abusive          0
HS_Individual    0
HS_Group         0
HS_Religion      0
HS_Race          0
HS_Physical      0
HS_Gender        0
HS_Other         0
HS_Weak          0
HS_Moderate      0
HS_Strong        0
dtype: int64


In [180]:
tweet_data_cleaned.to_csv('data_clean.csv', index=False)
print(f"Jumlah baris data setelah pembersihan: {len(tweet_data_cleaned)}")

Jumlah baris data setelah pembersihan: 13044


In [181]:
tweet_data = pd.read_csv("data_clean.csv", encoding='latin-1')
tweet_data.shape

(13044, 13)

### CASE FOLDING 

In [182]:
# konversi ke huruf kecil
tweet_data['Tweet_lower'] = tweet_data['Tweet'].str.lower()
tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Tweet_lower
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,- disaat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,rt user: user siapa yang telat ngasih tau elu?...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,"41. kadang aku berfikir, kenapa aku tetap perc..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,user user aku itu aku\n\nku tau matamu sipit t...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,user user kaum cebong kapir udah keliatan dong...


### FILTERING

In [183]:
# hapus karakter khusus
def remove_tweet_special (text) :
    text = str(text).replace("\\t", " ").replace("\\n", " ").replace("\\u", " ").replace("\\", "")
    text = str(text).encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",text).split())
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_lower'].apply(remove_tweet_special)

# hapus tanda baca
def remove_punctuation (text) :
    text = text.translate(str.maketrans("","",string.punctuation))
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_punctuation)

# hapus spasi putih di awal dan akhir
def remove_whitespace_LT(text):
    return text.strip()

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_whitespace_LT)

# hapus spasi berlebihan
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_whitespace_multiple)

# hapus karakter tunggal
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_single_char)

# hapus karakter yang tidak perlu
def remove_unnecessary_char(text):
    text = re.sub('\n', ' ', text)  # Remove '\n'
    text = re.sub('rt', ' ', text)  # Remove retweet symbol
    text = re.sub(r'\bRT\b(?!\w)', ' ', text)  # Remove standalone 'RT' only
    text = re.sub(r'\bUSER\b', ' ', text)  # Remove standalone 'USER'
    text = re.sub(r'\bURL\b', ' ', text)  # Remove standalone 'USER'
    text = re.sub(r'\buser\b', ' ', text)  # Remove standalone 'USER'
    text = re.sub('((@[^\s]+)|(#[^\s]+))', ' ', text)  # Remove @ and #
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', text)  # Remove URLs
    text = re.sub('  +', ' ', text)  # Remove extra spaces
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_unnecessary_char)

# hapus karakter non-alfanumerik dan angka
def remove_non_alphanumeric(text):
    text = re.sub(r'[0-9]+', '', text)  
    text = re.sub(r'[^a-zA-Z]+', ' ', text)  
    return text.strip()  


tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_non_alphanumeric)

# perluas singkata, word" menjadi word-word
def expand_contractions(text):
    text = re.sub(r'(\w+)(\d)', r'\1-\1', text)
    text = re.sub(r'(\w+)"', r'\1-\1', text)
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(expand_contractions)

# mengurangi pemnyebutan karakter berulang
def collapse_multiple_mentions(text):
    text = re.sub(r'(user\s+)+', 'user', text).strip()
    text = re.sub(r'(rt\s+)+', 'rt ', text).strip()
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(collapse_multiple_mentions)

# mengurangi karakter yang diulang lebih dari dua kali menjadi dua kali
def reduce_repeated_characters(text):
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(reduce_repeated_characters)

def classify(hs):
    retval = ""
    if int(hs) == 1:
        retval = 'negative'
    else:
        retval = 'positive'
    return retval

tweet_data['HS_class'] = tweet_data['HS'].apply(classify)

tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering,HS_class
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,negative
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,rt user: user siapa yang telat ngasih tau elu?...,siapa yang telat ngasih tau eluedan sarap gue ...,positive
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya p...,positive
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,user user aku itu aku\n\nku tau matamu sipit t...,aku itu aku ku tau matamu sipit tapi diliat da...,positive
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,user user kaum cebong kapir udah keliatan dong...,kaum cebong kapir udah keliatan dongoknya dari...,negative


### TOKENIZING

In [184]:
# memecah teks menjadi lebih kecil "token"
def word_tokenize_wrapper(text):
    return word_tokenize(text)

tweet_data["Tweet_token"] = tweet_data["Tweet_filtering"].apply(word_tokenize_wrapper)
tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering,HS_class,Tweet_token
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,negative,"[disaat, semua, cowok, berusaha, melacak, perh..."
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,rt user: user siapa yang telat ngasih tau elu?...,siapa yang telat ngasih tau eluedan sarap gue ...,positive,"[siapa, yang, telat, ngasih, tau, eluedan, sar..."
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya p...,positive,"[kadang, aku, berfikir, kenapa, aku, tetap, pe..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,user user aku itu aku\n\nku tau matamu sipit t...,aku itu aku ku tau matamu sipit tapi diliat da...,positive,"[aku, itu, aku, ku, tau, matamu, sipit, tapi, ..."
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,user user kaum cebong kapir udah keliatan dong...,kaum cebong kapir udah keliatan dongoknya dari...,negative,"[kaum, cebong, kapir, udah, keliatan, dongokny..."


In [185]:
tweet_data["Tweet_token"][0]

['disaat',
 'semua',
 'cowok',
 'berusaha',
 'melacak',
 'perhatian',
 'gue',
 'loe',
 'lantas',
 'remehkan',
 'perhatian',
 'yg',
 'gue',
 'kasih',
 'khusus',
 'ke',
 'elo',
 'basic',
 'elo',
 'cowok',
 'bego']

### REMOVE STOPWORD

In [186]:
# menghilangkan kata-kata umum yang sering muncul dan tidak memberikan banyak informasi khusus
stop_words = stopwords.words('indonesian')

def stopwords_removal(words):
    return [word for word in words if word not in stop_words]

tweet_data['Tweet_stopword'] = tweet_data['Tweet_token'].apply(stopwords_removal)

print('Stopword Removal :')
print(tweet_data['Tweet_stopword'].head())

Stopword Removal :
0    [disaat, cowok, berusaha, melacak, perhatian, ...
1    [telat, ngasih, tau, eluedan, sarap, gue, berg...
2    [kadang, berfikir, percaya, tuhan, jatuh, berk...
3                     [ku, tau, matamu, sipit, diliat]
4    [kaum, cebong, kapir, udah, keliatan, dongokny...
Name: Tweet_stopword, dtype: object


### NORMALIZATION

In [187]:
# normalisasi singkatan
normalize_word = pd.read_csv('kamus_singkatan.csv', encoding='latin-1')

normalize_word_dict = {}

for index, row in normalize_word.iterrows():
    if row.iloc[0] not in normalize_word_dict:
        normalize_word_dict[row.iloc[0]] = row.iloc[1] 

def normalized_term(document):
    return [normalize_word_dict[term] if term in normalize_word_dict else term for term in document]

tweet_data['Tweet_normalized'] = tweet_data['Tweet_stopword'].apply(normalized_term)

print('Normalized Term :')
print(tweet_data['Tweet_normalized'].head())

Normalized Term :
0    [di saat, cowok, berusaha, melacak, perhatian,...
1    [telat, memberi, tau, eluedan, sarap, gue, ber...
2    [kadang, berpikir, percaya, tuhan, jatuh, berk...
3                    [ku, tau, matamu, sipit, dilihat]
4    [kaum, cebong, kafir, sudah, kelihatan, dongok...
Name: Tweet_normalized, dtype: object


In [188]:
tweet_data["Tweet_normalized"][0]

['di saat',
 'cowok',
 'berusaha',
 'melacak',
 'perhatian',
 'gue',
 'kamu',
 'lantas',
 'remehkan',
 'perhatian',
 'yang',
 'gue',
 'kasih',
 'khusus',
 'kamu',
 'basic',
 'kamu',
 'cowok',
 'bego']

### STEMMING

In [189]:
# menghilangkan infleksi atau afiksasi dari kata kata menjadi bentuk dasar
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in tweet_data['Tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print('Total distinct stemmed term :',len(term_dict))

Total distinct stemmed term : 22520


In [190]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,':',term_dict[term])

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

tweet_data['Tweet_stemmed'] = tweet_data['Tweet_normalized'].apply(get_stemmed_term)

di saat : di saat
cowok : cowok
berusaha : usaha
melacak : lacak
perhatian : perhati
gue : gue
kamu : kamu
lantas : lantas
remehkan : remeh
yang : yang
kasih : kasih
khusus : khusus
basic : basic
bego : bego
telat : telat
memberi : beri
tau : tau
eluedan : eluedan
sarap : sarap
bergaul : gaul
cigax : cigax
jifla : jifla
calis : cal
itu : itu
licew : licew
kadang : kadang
berpikir : pikir
percaya : percaya
tuhan : tuhan
jatuh : jatuh
berkalikali : berkalikali
meninggalkan : tinggal
orang tuaku : orang tua
berencana : rencana
berpisah : pisah
kakakku : kakak
memilih : pilih
kristen : kristen
anak : anak
ter : ter
ku : ku
matamu : mata
sipit : sipit
dilihat : lihat
kaum : kaum
cebong : cebong
kafir : kafir
sudah : sudah
kelihatan : lihat
dongoknya : dongok
dungu : dungu
haha : haha
ya : ya
bani : bani
taplak : taplak
dan kawan kawan : dan kawan kawan
xfxfxxfxfxxfxfxx : xfxfxxfxfxxfxfxx
deklarasi : deklarasi
pilihan kepala daerah : pilih kepala daerah
aman : aman
anti : anti
hoaks : hoaks


In [191]:
def remove_punc(text):
    text = " ".join([char for char in text if char not in string.punctuation])
    return text

tweet_data['Tweet_stemmed_join'] = tweet_data['Tweet_stemmed'].apply(lambda x: remove_punc(x))

tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,...,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering,HS_class,Tweet_token,Tweet_stopword,Tweet_normalized,Tweet_stemmed,Tweet_stemmed_join
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,...,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,negative,"[disaat, semua, cowok, berusaha, melacak, perh...","[disaat, cowok, berusaha, melacak, perhatian, ...","[di saat, cowok, berusaha, melacak, perhatian,...","[di saat, cowok, usaha, lacak, perhati, gue, k...",di saat cowok usaha lacak perhati gue kamu lan...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,...,0,0,rt user: user siapa yang telat ngasih tau elu?...,siapa yang telat ngasih tau eluedan sarap gue ...,positive,"[siapa, yang, telat, ngasih, tau, eluedan, sar...","[telat, ngasih, tau, eluedan, sarap, gue, berg...","[telat, memberi, tau, eluedan, sarap, gue, ber...","[telat, beri, tau, eluedan, sarap, gue, gaul, ...",telat beri tau eluedan sarap gue gaul cigax ji...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,...,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",kadang aku berfikir kenapa aku tetap percaya p...,positive,"[kadang, aku, berfikir, kenapa, aku, tetap, pe...","[kadang, berfikir, percaya, tuhan, jatuh, berk...","[kadang, berpikir, percaya, tuhan, jatuh, berk...","[kadang, pikir, percaya, tuhan, jatuh, berkali...",kadang pikir percaya tuhan jatuh berkalikali k...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,...,0,0,user user aku itu aku\n\nku tau matamu sipit t...,aku itu aku ku tau matamu sipit tapi diliat da...,positive,"[aku, itu, aku, ku, tau, matamu, sipit, tapi, ...","[ku, tau, matamu, sipit, diliat]","[ku, tau, matamu, sipit, dilihat]","[ku, tau, mata, sipit, lihat]",ku tau mata sipit lihat
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,...,1,0,user user kaum cebong kapir udah keliatan dong...,kaum cebong kapir udah keliatan dongoknya dari...,negative,"[kaum, cebong, kapir, udah, keliatan, dongokny...","[kaum, cebong, kapir, udah, keliatan, dongokny...","[kaum, cebong, kafir, sudah, kelihatan, dongok...","[kaum, cebong, kafir, sudah, lihat, dongok, du...",kaum cebong kafir sudah lihat dongok dungu haha


Setelah dilakukan preprocessing, data selanjutnya dimasukkan ke dalam file csv `preprocessing_Data.csv`

In [192]:
tweet_data.to_csv('preprocessing_data.csv',encoding='utf8', index=False)